# PowerShell Debugging Examples
Like with any proper debugger, via the PowerShell debugger, we can step into statements, over them, out of them, can inspect variables, test logic, do whatever we must to solve the issue, all with the active running code in "suspended" mode, advancing it how and when we want.  For info about leveraging the debugger, use `?` or `h` within the debugger, or view the PowerShell help for [about_Debugger](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_debugger)

The examples here are meant to be walked through by the user in their own PowerShell session.  The examples:

|  Item  | Description |
|:--------|:-------------|
[Unexpected Logic Result](#Unexpected-Logic-Result) | In this example, there is the need to determine why some logic set is returning `$false` instead of the expected `$true`
[Null value vs Empty String](#Null-value-vs-Empty-String) | Trying to test for properties with seemingly no value, but getting unexpected results

## Unexpected Logic Result
In this sample, there is the need to determine why some logic set is returning `$false` instead of the expected `$true`. We can use the PowerShell debugger for some debugging

In [18]:
## some user data to use later (JSON file is in this directory)
$arrUsersInfo = Get-Content sampleUserInfo.json | ConvertFrom-Json
## see what's inc that data
$arrUsersInfo


DisplayName SamAccountName PhoneNbr
----------- -------------- --------
Billy Boy   bboy0001       800-123-4567
Suzy Queue  squeue0009     800-234-5678
Ricky Rocky rrocky0045     



Our code is intended to iterate over the user objects, and for any object that has a phone number value, it would take action (for demonstration purposes here, we are just using `Write-Verbose` to write out a verbose message to the PowerShell console)

In [25]:
## iterate over users, taking certain actions when particular things are present (in real life, we'd do something more than Write-Verbose, of course)
$arrUsersInfo | Foreach-Object {
    $oThisUser = $_
    Write-Verbose -Verbose "Working on user '$($oThisUser.DisplayName)'"
    Write-Verbose -Verbose $(if ($oThisUser.PhoneNmbr -match "\d{3}-\d{3}-\d{4}") {"Updating phone number in directory!"} else {"Phone number for this cat not in expected format -- taking no action"})
}

VERBOSE: Working on user 'Billy Boy'
VERBOSE: Phone number for this cat not in expected format -- taking no action
VERBOSE: Working on user 'Suzy Queue'
VERBOSE: Phone number for this cat not in expected format -- taking no action
VERBOSE: Working on user 'Ricky Rocky'
VERBOSE: Phone number for this cat not in expected format -- taking no action


Hmm -- we would have expected the first two users to have some action taken, since they have a seemingly valid phone number in the data, and only the last user to have the "taking no action" scenario, as that is the user with no phone number.  What's the deal?

We can step through the logic and inspect things in real time by employing the PowerShell debugger.  Generally we would do so by a "breakpoint" on a script or command. We'll do it here by setting a breakpoint for when a particular variable gets read.

In [ ]:
## set a breakpoint for when the given variable gets read
Set-PSBreakpoint -Variable arrUsersInfo -Mode Read
## then, let's issue the same script
## iterate over users, taking certain actions when particular things are present (in real life, we'd do something more than Write-Verbose, of course)
$arrUsersInfo | Foreach-Object {
    $oThisUser = $_
    Write-Verbose -Verbose "Working on user '$($oThisUser.DisplayName)'"
    Write-Verbose -Verbose $(if ($oThisUser.PhoneNmbr -match "\d{3}-\d{3}-\d{4}") {"Updating phone number in directory!"} else {"Phone number for this cat not in expected format -- taking no action"})
}

At this point, we will enter the PowerShell debugger, and can follow the code as desired to figure out what the world is causing this unexpected behavior

## Null value vs Empty String
In this sample, we should figure out how we can determine if a given property's value is essentially empty (when we are not yet sure if it is actually `$null`, or just an empty string, or maybe whitespace characters only)

In [26]:
## again, some user data to use later (JSON file is in this directory)
$arrUsersInfo = Get-Content sampleUserInfo.json | ConvertFrom-Json
## see what's inc that data
$arrUsersInfo


DisplayName SamAccountName PhoneNbr
----------- -------------- --------
Billy Boy   bboy0001       800-123-4567
Suzy Queue  squeue0009     800-234-5678
Ricky Rocky rrocky0045     



Now let's see our code that is trying to take action if the user info object has a phone number:

In [27]:
## take action if the phone number is present (not $null)
$arrUsersInfo | Foreach-Object {
    $oThisUser = $_
    Write-Verbose -Verbose "Working on user '$($oThisUser.DisplayName)'"
    Write-Verbose -Verbose $(if ($null -ne $oThisUser.PhoneNbr) {"Updating phone number in directory!"} else {"Phone number for this cat not in expected format -- taking no action"})
}

VERBOSE: Working on user 'Billy Boy'
VERBOSE: Updating phone number in directory!
VERBOSE: Working on user 'Suzy Queue'
VERBOSE: Updating phone number in directory!
VERBOSE: Working on user 'Ricky Rocky'
VERBOSE: Updating phone number in directory!


What the -- we would have expected _only_ the first two users to have some action taken, but that the last user to have the "taking no action" scenario, as that is the user with no phone number.  What's the deal?

Once again, we can step through the logic and inspect things in real time via the PowerShell debugger. Again we'll do so by a "breakpoint" for when a particular variable gets read.

In [ ]:
## set a breakpoint for when the given variable gets read
Set-PSBreakpoint -Variable arrUsersInfo -Mode Read

## take action if the phone number is present (not $null)
$arrUsersInfo | Foreach-Object {
    $oThisUser = $_
    Write-Verbose -Verbose "Working on user '$($oThisUser.DisplayName)'"
    Write-Verbose -Verbose $(if ($null -ne $oThisUser.PhoneNbr) {"Updating phone number in directory!"} else {"Phone number for this cat not in expected format -- taking no action"})
}

Once again, we will enter the PowerShell debugger, and can follow the code as desired to figure out what the world is causing this unexpected behavior. Hint: empty string is different than `$null`!